In [ ]:
!pip install scikit-optimize
from keras import Input, Model
from keras.layers import Conv3D, MaxPooling3D, Conv3DTranspose, ConvLSTM3D
import os
import cv2
import glob
from skopt import BayesSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from tensorflow.python.keras import backend as K
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import random
import shutil
from sklearn.model_selection import train_test_split
from skimage.metrics import structural_similarity as compare_ssim
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    average_precision_score,
    precision_score,
    recall_score,
    f1_score
)



#Comment this out if You can use GPU instead of CPU

# Set the TensorFlow session to use GPU
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
K.set_session(sess)



#PARAMETERS
num_of_cv_tuning = 5
height, width = 180, 320
desired_no_channels = 3
batch_size = 2
max_frames = 200
max_videos = 4
max_videos_eval = 3
epochs=5
iterations = 20
window_size=8
threshold = 0.5
num_windows = max_frames // window_size
input_shape = (window_size, height, width, desired_no_channels) #num_widnows,
anomaly_label = "anomaly"
split_train_val_ratio = 0.5
model_filepath = '/content/drive/MyDrive/Models/autoencoder.h5'
from keras.utils import Sequence

#HYPERPARAMETERS
param_space = {
    'batch_size': (2, 4),
}


class DataGenerator(Sequence):
    def __init__(self, windowed_optical_flows, batch_size, split_train_val_ratio):
        self.windowed_optical_flows = windowed_optical_flows
        self.batch_size = batch_size
        self.split_train_val_ratio = split_train_val_ratio

    def __len__(self):
        return int(np.ceil(len(self.windowed_optical_flows) / self.batch_size))

    def __getitem__(self, idx):
        batch_samples = self.windowed_optical_flows[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_samples

    def get_X_val(self):
        split_idx = int(len(self.windowed_optical_flows) * self.split_train_val_ratio)
        return np.concatenate(self.windowed_optical_flows[:split_idx])

    def get_X_train(self):
        split_idx = int(len(self.windowed_optical_flows) * self.split_train_val_ratio)
        return np.concatenate(self.windowed_optical_flows[split_idx:-1])

    def get_X_test(self):
        return self.windowed_optical_flows[-1]

def train_model_incremental_with_validation(model, filename, train_paths, max_videos, batch_size, epochs, split_train_val_ratio, iterations, load_weights=False):
    start_video = ""
    total_anomaly_scores = []
    data_generator = None
    for iteration in tqdm(range(iterations), desc="Iteratons"):
      # Load the previously trained model if it exists
      if os.path.exists(model_filepath) and load_weights:
          model.load_weights(model_filepath)
      load_weights = True
      windowed_optical_flows, _, start_video = get_windowed_optical_flows_data(train_paths, max_videos, window_size, start_video)
      if len(windowed_optical_flows) == 0:
          print("No more data available, break the iteration loop")
          break
      if data_generator is None:
          data_generator = DataGenerator(windowed_optical_flows, batch_size, split_train_val_ratio)
      else:
          data_generator.windowed_optical_flows = windowed_optical_flows

      x_train = data_generator.get_X_train()
      x_val = data_generator.get_X_val()
      x_test = data_generator.get_X_test()

      # Perform Bayesian optimization to find optimal hyperparameters
      model = set_best_hyperparameters(build_lstm_autoencoder, param_space, x_val, scoring_func)
      print("X_train shape {}".format(x_train.shape))
      model.fit(x_train, x_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, x_test))

      # Evaluate the model on the validation set
      total_anomaly_scores.append(validate_model(x_test, model))

      del windowed_optical_flows
      del x_train, x_val, x_test
      # Save the updated model
      model.save(model_filepath)
    return total_anomaly_scores


def resize_frames(frames, target_size):
    resized_frames = []
    for frame in frames:
        frame_data = cv2.imread(frame)
        if frame_data is None:
            print(f"Error loading frame: {frame}")
            continue
        resized_frame = cv2.resize(frame_data, target_size, interpolation=cv2.INTER_LINEAR)
        resized_frames.append(resized_frame)
    return resized_frames


def get_movie_chunk(path, max_frames, start_frame=0):
    frame_files = sorted(file for file in os.listdir(path) if file.endswith('.tif'))
    num_frames = len(frame_files)
    if start_frame >= num_frames:
        return None
    end_frame = min(start_frame + max_frames, num_frames)
    frame_files = frame_files[start_frame:end_frame]
    resized_frames = resize_frames([os.path.join(path, file) for file in frame_files], (320, 180))
    return np.array(resized_frames)


def get_movie(path, max_frames):
    frame_files = sorted(file for file in os.listdir(path) if file.endswith('.tif'))
    num_frames = len(frame_files)
    if num_frames > 0:
      # Duplicate frames if the number of frames is smaller than max_frames
      if num_frames < max_frames:
          duplication_factor = max_frames // num_frames
          remaining_frames = max_frames % num_frames
          duplicated_frames = frame_files * duplication_factor + frame_files[:remaining_frames]
          frame_files += duplicated_frames
      elif num_frames > max_frames:
          frame_files = frame_files[:max_frames]
      else:
          frame_files = frame_files
      resized_frames = resize_frames([os.path.join(path, file) for file in frame_files], (320, 180))
    else:
      resized_frames = []
    return np.array(resized_frames)


def get_data(paths, max_videos, start_video):
    dataset = []
    labels = []
    videos_processed = 0
    current_path_index = 0
    video_paths = []
    # Collect all video paths from all directories
    for dataset_path in paths:
        video_paths.extend(sorted([os.path.join(dataset_path, video_directory) for video_directory in os.listdir(dataset_path)]))
    # Find the index of the start_video in the video paths
    if start_video:
        try:
            start_video_index = video_paths.index(start_video)
            current_path_index = start_video_index // max_videos
            video_paths = video_paths[start_video_index:]
        except ValueError:
            pass
    for video_path in video_paths:
        if videos_processed >= max_videos:
            break  # Stop processing videos once the desired number is reached
        video = get_movie(video_path, max_frames)
        print(f"Video path: {video_path}")
        if video is None or len(video) == 0:
            continue
        dataset.append(video)
        video_labels = extract_labels_from_frames(video_path, anomaly_label)
        labels.extend(video_labels)
        videos_processed += 1
    dataset = [video for video in dataset if video.shape == (max_frames, height, width, desired_no_channels)]
    dataset = np.stack(dataset) if len(dataset) > 0 else []
    if start_video == "":
        next_start_video = video_paths[max_videos] if len(video_paths) > max_videos else None
    elif current_path_index + 1 < len(paths):
        next_start_video = os.path.join(paths[current_path_index + 1], os.listdir(paths[current_path_index + 1])[0])
    else:
        next_start_video = None
    return dataset, labels, next_start_video

def extract_labels_from_frames(video_path, substring_to_find):
    frame_names = [os.path.basename(frame_path) for frame_path in sorted(glob.glob(os.path.join(video_path, "*.tif")))]
    labels = [1 if substring_to_find in frame_name else 0 for frame_name in frame_names]
    return labels


def calculate_max_dimensions(paths):
    max_height, max_width, max_frames = 0, 0, 0
    for dataset_path in paths:
        for video_directory in os.listdir(dataset_path):
            video_path = os.path.join(dataset_path, video_directory)
            frame_files = [file for file in os.listdir(video_path) if file.endswith('.tif')]
            max_frames = max(max_frames, len(frame_files))
            for frame_file in frame_files:
                frame = cv2.imread(os.path.join(video_path, frame_file), cv2.IMREAD_UNCHANGED)
                if frame is None:
                    print(f"Error loading frame: {frame_file}")
                    continue
                max_height, max_width = max(max_height, frame.shape[0]), max(max_width, frame.shape[1])
    return max_height, max_width, max_frames


def generate_optical_flow(frame_files, max_height, max_width):
    # Create an empty array to store the optical flow map volume
    optical_flow_volume = np.zeros((len(frame_files), max_height, max_width, desired_no_channels), dtype=np.float32)
    #print("Shape of a frame: {}".format(frame_files[0].shape))
    prev_gray = cv2.cvtColor(frame_files[0], cv2.COLOR_BGR2GRAY)
    for frame_index in range(1, len(frame_files)):
        frame = frame_files[frame_index]
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        # Store the optical flow in the optical flow map volume
        optical_flow_volume[frame_index - 1, :, :, 0] = flow[..., 0]
        optical_flow_volume[frame_index - 1, :, :, 1] = flow[..., 1]
        # Convert the flow vectors to magnitude and angle
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        # Normalize the magnitude to the range [0, 255]
        magnitude_normalized = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
        # Convert the angle to the hue channel in the HSV color space
        hsv = np.zeros_like(frame)
        hsv[..., 0] = angle * (180 / np.pi) / 2  # Hue channel
        hsv[..., 1] = 255  # Saturation channel
        hsv[..., 2] = magnitude_normalized  # Value channel
        # Convert HSV to BGR color representation
        rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        # Set the third channel with RGB color values
        optical_flow_volume[frame_index - 1, :, :, 2] = rgb[..., 2]
        prev_gray = gray
    # return flow_volume_map_for_a_video
    return optical_flow_volume


def calculate_optical_flows(data, max_frame_height, max_frame_width):
    optical_flows = []
    for video in data:
        optical_flow_video = generate_optical_flow(video, max_frame_height, max_frame_width)
        optical_flows.append(optical_flow_video)
    optical_flows_array = np.stack(optical_flows)
    print(f"dataset shape: {optical_flows_array.shape}")
    return optical_flows_array

def update_maximum_dimensions(max_frames, max_height, max_width, optical_flow_video):
    # Update the maximum dimensions
    max_frames = max(max_frames, optical_flow_video.shape[0])
    print("Max frames: {} ".format(max_frames))
    max_height = max(max_height, optical_flow_video.shape[1])
    max_width = max(max_width, optical_flow_video.shape[2])
    channels = optical_flow_video.shape[3]
    return channels, max_frames, max_height, max_width


def divide_into_windows(optical_flow_dataset, window_size):
    windowed_dataset = []
    for optical_flow_video in optical_flow_dataset:
        windows = divide_into_windows_video(optical_flow_video, window_size)
        print(f"Window shape: {windows.shape}")
        windowed_dataset.append(windows)
    return windowed_dataset


def divide_into_windows_video(video, window_size):
    num_frames = video.shape[0]
    windows = [video[i:i + window_size] for i in range(0, num_frames, window_size)]
    compatible_windows = [window for window in windows if window.shape == (window_size, height, width, desired_no_channels)]
    np_windows = np.stack(compatible_windows)
    return np_windows


def divide_data_into_dirs(moved_from_directory, moved_to_directory, split_ratio):
  subdirectories = os.listdir(moved_from_directory)
  random.shuffle(subdirectories)
  n_subdirectories = len(subdirectories)
  n_validation = int(split_ratio * n_subdirectories)
  n_test = n_subdirectories - n_validation
  if not os.path.exists(moved_to_directory):
      os.makedirs(moved_to_directory)
  validation_subdirectories = subdirectories[:n_validation]
  test_subdirectories = subdirectories[n_validation:]
  for subdirectory in validation_subdirectories:
      source = os.path.join(moved_from_directory, subdirectory)
      destination = os.path.join(moved_to_directory, subdirectory)
      shutil.move(source, destination)
  # Rename "Test" substring to "Validation" in validation directory
  for root, dirs, files in os.walk(moved_to_directory):
      for dir_name in dirs:
          if "Test" in dir_name:
              new_dir_name = dir_name.replace("Test", "Validation")
              os.rename(os.path.join(root, dir_name), os.path.join(root, new_dir_name))
  print(f"Moved {n_validation} subdirectories to the validation directory: {moved_to_directory}.")
  print(f"Remaining subdirectories in the test directory: {n_test}.")


# Define the model architecture
def build_lstm_autoencoder():
    input_data = Input(shape=input_shape)
    encoded = Conv3D(filters=128, kernel_size=(10, 10, 3), activation='relu', padding='same', strides=(2, 2, 1))(input_data)
    print("Shape in the after first conv3D layer {}".format(encoded.shape))
    encoded = Conv3D(filters=64, kernel_size=(6, 6, 3), activation='relu', padding='same', strides=(2, 2, 1))(encoded)
    print("Shape in the after second conv3D layer {}".format(encoded.shape))
    encoded = ConvLSTM3D(filters=64, kernel_size=(3, 3, 3), padding='same', return_sequences=True)(encoded[:, None, ...])
    print("Shape in the after first ConvLSTM layer {}".format(encoded.shape))
    encoded = ConvLSTM3D(filters=32, kernel_size=(3, 3, 3), padding='same', return_sequences=True)(encoded)
    print("Shape in the after second ConvLSTM layer {}".format(encoded.shape))
    encoded = ConvLSTM3D(filters=64, kernel_size=(3, 3, 3), padding='same', return_sequences=True)(encoded)
    print("Shape in the after third ConvLSTM layer {}".format(encoded.shape))
    # Decoder
    decoded = Conv3DTranspose(filters=128, kernel_size=(6, 6, 3), strides=(2, 2, 1), padding='same', activation='relu')(encoded[:, 0, ...])
    print("Shape in the after first Conv3DTranspose layer {}".format(decoded.shape))
    decoded = Conv3DTranspose(filters=1, kernel_size=(10, 10, 3), strides=(2, 2, 1), padding='same', activation='relu')(decoded)
    print("Shape in the after second Conv3DTranspose layer {}".format(decoded.shape))
    decoded = Conv3D(filters=3, kernel_size=(3, 3, 3), activation='sigmoid', padding='same')(decoded)
    print("Shape in the after third Conv3DTranspose layer {}".format(decoded.shape))
    model = Model(input_data, decoded)
    model.compile(optimizer="adam", loss="mse")
    # Autoencoder model
    return model


def assign_window_labels(windowed_optical_flows, frame_labels):
    window_labels = []
    for idx, window_frames in enumerate(windowed_optical_flows):
        window_label = 0  # Initialize the window label as normal (0)
        for frame in window_frames:
            if frame_labels[idx] == 1:  # If any frame in the window is abnormal
                window_label = 1  # Set the window label as abnormal (1)
                break
        window_labels.append(window_label)
    return window_labels


def get_windowed_optical_flows_data(paths, max_videos, window_size, start_video):
  x_train, frame_labels , start_video = get_data(paths, max_videos, start_video)
  if len(x_train) == 0:
    return [], [], ""
  print(f"Shape of training data: {x_train.shape}")
  print(f"Shape of one video: {np.array(x_train[0]).shape}")
  optical_flows = calculate_optical_flows(x_train, 180, 320)
  if len(optical_flows) == 0:
   return [], [], ""
  windowed_optical_flows = divide_into_windows(optical_flows, window_size)
  if len(windowed_optical_flows) == 0:
    return [], [], ""
  # Assign window-level labels based on frame-level labels
  window_labels = assign_window_labels(windowed_optical_flows, frame_labels)
  del optical_flows
  return windowed_optical_flows, window_labels, start_video


def set_best_hyperparameters(model_fn, param_space, X_val, custom_scoring_func):
    # Wrap the Keras model in a scikit-learn compatible wrapper
    wrapped_model = KerasRegressor(build_fn=model_fn)
    n_splits = min(3, len(X_val)//2)  # Set the number of splits based on the number of samples
    opt = BayesSearchCV(wrapped_model, param_space, scoring=custom_scoring_func, cv=n_splits, n_jobs=1)
    # Perform Bayesian optimization to find optimal hyperparameters
    print(f"X_val shape == {X_val.shape}")
    opt.fit(X_val, X_val)
    # Get the best hyperparameters found by Bayesian optimization
    best_hyperparams = opt.best_params_
    # Unwrap the model
    best_model = opt.best_estimator_.model
    # Update the model with the best hyperparameters
    best_model.set_params(**best_hyperparams)
    # Delete unused variables to free up memory
    del wrapped_model
    del opt
    return best_model


def calculate_ms_ssim_error(original_flow, reconstructed_flow):
    ms_ssim_error = compare_ssim(original_flow, reconstructed_flow, multichannel=True)
    return ms_ssim_error


def calculate_anomaly_score(original_flow, reconstructed_flow):
  ms_msim_error = calculate_ms_ssim_error(original_flow, reconstructed_flow)
  return 1 - ms_msim_error


def normalize_scores(scores):
    scaler = MinMaxScaler()
    normalized_scores = scaler.fit_transform(scores.reshape(-1, 1))
    return normalized_scores.flatten()


def scoring_func(model, data, y):
    reconstructed_flows = model.predict(data)
    # Calculate the anomaly scores
    anomaly_scores = calculate_anomaly_score(data, reconstructed_flows)
    if anomaly_scores:
        # Apply your chosen normalization technique here
        normalized_scores = normalize_scores(anomaly_scores)
    else:
        normalized_scores = anomaly_scores
    return -np.mean(normalized_scores)


def evaluate_model(model, filename, paths, max_frames, threshold, max_videos, start_video, readFromFile=False):
  if readFromFile:
    # Load the trained model
    if os.path.exists(model_filepath):
        model = tf.models.load_model(model_filepath)
    else:
        print("Trained model weights not found!")
        return
  total_anomaly_scores = []
  true_labels = []
  windowed_optical_flows, window_labels, start_video = get_windowed_optical_flows_data(paths, max_videos, window_size, start_video)
  # Predict the reconstructed optical flows
  reconstructed_optical_flows = model.predict(windowed_optical_flows)
  # Calculate the anomaly scores for each reconstructed flow
  anomaly_scores = []
  for i in range(len(reconstructed_optical_flows)):
      original_flow = windowed_optical_flows[i]
      reconstructed_flow = reconstructed_optical_flows[i]
      anomaly_score = calculate_anomaly_score(original_flow, reconstructed_flow)
      anomaly_scores.append(anomaly_score)
  # Collect the anomaly scores and true labels for all videos
  total_anomaly_scores.extend(anomaly_scores)
  true_labels.extend(window_labels)  # Assuming `labels` is a list of true anomaly labels
  # Convert lists to numpy arrays for evaluation metrics computation
  total_anomaly_scores = np.array(total_anomaly_scores)
  true_labels = np.array(true_labels)
  # Compute precision, recall, and F1-score
  precision = precision_score(true_labels, total_anomaly_scores > threshold)
  recall = recall_score(true_labels, total_anomaly_scores > threshold)
  f1 = f1_score(true_labels, total_anomaly_scores > threshold)
  # Print the evaluation metrics
  print("Precision:", precision)
  print("Recall:", recall)
  print("F1-score:", f1)
  # Plot the anomaly scores
  plt.plot(total_anomaly_scores)
  plt.xlabel("Frame Index")
  plt.ylabel("Anomaly Score")
  plt.title("Anomaly Detection")
  plt.show()
  return start_video


def validate_model(X_val, model):
  reconstructed_optical_flows = model.predict(X_val)
  # Calculate the anomaly scores for each reconstructed flow
  anomaly_scores = []
  for i in range(len(reconstructed_optical_flows)):
      original_flow = X_val[i]
      reconstructed_flow = reconstructed_optical_flows[i]
      anomaly_score = calculate_anomaly_score(original_flow, reconstructed_flow)
      anomaly_scores.append(anomaly_score)
  plot_anomaly_score_distribution(anomaly_scores)
  threshold = determine_threshold(anomaly_scores)
  plot_anomaly_score_trend(anomaly_scores, threshold)
  return anomaly_score



def plot_anomaly_score_distribution(anomaly_scores):
  plt.hist(anomaly_scores, bins=20, density=True, alpha=0.5)
  plt.xlabel('Anomaly Score')
  plt.ylabel('Density')
  plt.title('Distribution of Anomaly Scores')
  plt.show()


def plot_roc_curve(y_true, anomaly_scores):
    fpr, tpr, thresholds = roc_curve(y_true, anomaly_scores)
    roc_auc = auc(fpr, tpr)

    plt.plot(fpr, tpr, label='ROC curve (AUC = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.show()


def plot_optical_flow_comparison(original_flow, reconstructed_flow):
    # Plot the original and reconstructed flows side by side
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
    axes[0].imshow(original_flow)
    axes[0].set_title('Original Flow')
    axes[1].imshow(reconstructed_flow)
    axes[1].set_title('Reconstructed Flow')
    plt.show()

def plot_anomaly_score_trend(anomaly_scores, threshold):
    # Plot the anomaly scores over the dataset
    plt.plot(anomaly_scores)
    plt.axhline(y=threshold, color='r', linestyle='--', label='Threshold')
    plt.xlabel('Sample')
    plt.ylabel('Anomaly Score')
    plt.title('Anomaly Score Trend')
    plt.legend()
    plt.show()

def determine_threshold(total_anomaly_scores):
    # Calculate the mean and standard deviation of the total_anomaly_scores
    mean_score = np.mean(total_anomaly_scores)
    std_score = np.std(total_anomaly_scores)
    # Plot the histogram of total_anomaly_scores
    plot_anomaly_score_distribution(total_anomaly_scores)
    # Set the threshold as a multiple of the standard deviation from the mean
    threshold = mean_score + 2 * std_score
    print("Threshold for anomaly detection:", threshold)
    return threshold

if __name__ == "__main__":
    USCDped1_train_path = '/content/drive/MyDrive/UCSDped1/Train'
    USCDped2_train_path = '/content/drive/MyDrive/UCSDped2/Train'

    USCDped1_test_path = '/content/drive/MyDrive/UCSDped1/Test'
    USCDped2_test_path = '/content/drive/MyDrive/UCSDped2/Test'

    USCDped1_validation_path = '/content/drive/MyDrive/UCSDped1/Validation'
    USCDped2_validation_path = '/content/drive/MyDrive/UCSDped2/Validation'

    train_paths = [USCDped1_train_path, USCDped2_train_path]
    validation_paths = [USCDped1_validation_path, USCDped2_validation_path]

    lstm_autoencoder = build_lstm_autoencoder()
    total_anomaly_scores = train_model_incremental_with_validation(lstm_autoencoder, "autoencoder", train_paths, max_videos, batch_size, epochs, split_train_val_ratio, iterations)
